In [215]:
import pandas as pd
from datetime import date
from nsepy import get_history
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
# stocks = pd.read_parquet('/Users/entirety/Documents/Entirety-AMC/Data/NIFTY_50.parquet')

stocks=get_history(symbol="NIFTY",
                            start=date(2022,1,1),
                            end=date(2022,9,1),
                            index=True)

stocks

In [ ]:
df=stocks[["Close"]]
df

In [ ]:

df.plot(figsize=(12,8),title="Closing Price",fontsize=12)
plt.show()

In [ ]:
period_high=20
df["highest"]=(df["Close"].rolling(window=period_high).max()).shift()

In [ ]:
# df.loc.plot(figsize=(12,8),fontsize=12)
# plt.show()

In [ ]:
df.head


In [ ]:
df["Buy"]=np.where(df["Close"].shift()>df["highest"].shift(),1,0)


In [ ]:
period_low=40
df["lowest"]=(df["Close"].rolling(window=period_low).min()).shift()


df["Sell"]=np.where(df["Close"].shift()<df["lowest"].shift(),1,0)


df


In [ ]:
df_trades=df[(df.Buy==1) | (df.Sell==1)]
df_trades

In [ ]:

trade=df_trades[(df_trades.Buy.diff()==1)|(df_trades.Sell.diff()==1)]
trade

In [ ]:
df

In [ ]:

Buy_dates=[]
Sell_dates=[]
buys=[]
sells=[]
open_pos=False


# In[21]:


for i in range(len(df)):
    if df.Buy.iloc[i]:
        if open_pos==False:
            buys.append(i)
            open_pos=True
            Buy_dates.append(df.iloc[i].name)
    elif df.Sell.iloc[i]:
        if open_pos:
            sells.append(i)
            open_pos=False
            Sell_dates.append(df.iloc[i].name)
            


# In[22]:




In [ ]:

print(len(buys))
print(len(sells))

In [ ]:

if len(buys)>len(sells):
    buys.pop(-1)
    Buy_dates.pop(-1)

print(len(buys))
print(len(sells))



In [ ]:
check=pd.DataFrame({'buydate':Buy_dates,'selldate':Sell_dates,'buyprice':df.loc[Buy_dates].Close.values,'sellprice':df.loc[Sell_dates].Close.values})
check


In [ ]:

check["pnl_perc"]=(check.sellprice-check.buyprice)/check.buyprice
check



In [ ]:

check["cumm+pnl_perc"]=(check["pnl_perc"]+1).cumprod()
check


In [ ]:

totaltrades=len(check.pnl_perc)
totaltrades


In [ ]:

profits=df.loc[check.selldate].Close.values-df.loc[check.buydate].Close.values
profits


In [ ]:


i=0
profit_trades=len([i for i in profits if i>0])
profit_trades_perc=(profit_trades/totaltrades)*100

print("No of profit trades = {}".format(profit_trades))
print("Win ratio = {}".format(profit_trades_perc))

In [ ]:

profits_perc=(df.loc[check.selldate].Close.values-df.loc[check.buydate].Close.values)/df.loc[check.buydate].Close.values

avg_profit=profits_perc.mean()*100
avg_profit


In [ ]:

list_profits=(profits_perc+1).cumprod()
list_profits


In [ ]:

net_returns_perc=(list_profits[-1]-1)*100
net_returns_perc


In [ ]:

profits_perc.min()

In [ ]:

profits_perc.max()

In [ ]:

init_inv=1000
inv_ret=init_inv*(profits_perc+1).cumprod()
inv_ret[-1]


In [ ]:

def test_strategy(stock, buy_days, sell_days,invest):
    data=yf.download(stock)
    df=data.Close.to_frame()
    df["highest"]=(df["Close"].rolling(window=buy_days).max()).shift()
    df["Buy"]=np.where(df["Close"].shift()>df["highest"].shift(),1,0)
    df["lowest"]=(df["Close"].rolling(window=sell_days).min()).shift()
    df["Sell"]=np.where(df["Close"].shift()<df["lowest"].shift(),1,0)
    
    Buy_dates=[]
    Sell_dates=[]
    buys=[]
    sells=[]
    open_pos=False
    
    for i in range(len(df)):
        if df.Buy.iloc[i]:
            if open_pos==False:
                buys.append(i)
                open_pos=True
                Buy_dates.append(df.iloc[i].name)
        elif df.Sell.iloc[i]:
            if open_pos:
                sells.append(i)
                open_pos=False
                Sell_dates.append(df.iloc[i].name)
    
    if len(buys)>len(sells):
        buys.pop(-1)
        Buy_dates.pop(-1)
        
    check=pd.DataFrame({'buydate':Buy_dates,'selldate':Sell_dates,'buyprice':df.loc[Buy_dates].Close.values,'sellprice':df.loc[Sell_dates].Close.values})
    check["pnl_perc"]=(check.sellprice-check.buyprice)/check.buyprice
    check["cumm+pnl_perc"]=(check["pnl_perc"]+1).cumprod()

    profits_perc=(df.loc[check.selldate].Close.values-df.loc[check.buydate].Close.values)/df.loc[check.buydate].Close.values
    list_profits=(profits_perc+1).cumprod()
    net_returns_perc=(list_profits[-1]-1).round(3)*100

    inv_ret=init_inv*(profits_perc+1).cumprod()
    return_inv=inv_ret[-1]

    large_loss=profits_perc.min().round(3)*100
    large_profit=profits_perc.max().round(3)*100   


    return print("Returns perc= {}%".format(net_returns_perc)), print("Invested return= {}".format(return_inv)), print("Largest Loss= {}%".format(large_loss)), print("Largest Profit= {}%".format(large_profit))

In [ ]:
test_strategy("KO", 40, 40,1000)